# Fast Style Transfer

## Summary
In this notebook, I am going to use Fast Style Transfer from TFHub to generate monet-styled images. It can generate 7000 images in about 350 seconds, which takes 0.05 second per image.

## Import Packages

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import time
import os
from PIL import Image
import shutil

## Common Parameters

In [ ]:
base_path = "/kaggle/input/gan-getting-started"
content_path = os.path.join(base_path, "photo_jpg")
style_path = os.path.join(base_path, "monet_jpg")
output_path = "/kaggle/working/images"
image_size = 256
num_outputs = 7000

## Common Functions

In [ ]:
def sample(photos, count):
    photos = photos[np.random.choice(len(photos), count)]
    for photo in photos:
        img = tf.keras.preprocessing.image.load_img(photo)
        plt.imshow(img)
        plt.show()

## Import Datasets

In [ ]:
style_images = np.array([os.path.join(style_path, path) for path in os.listdir(style_path)])
np.random.shuffle(style_images)
print(style_images[:10])

In [ ]:
content_images = np.array([os.path.join(content_path, path) for path in os.listdir(content_path)])
np.random.shuffle(content_images)
print(content_images[:10])

## Exploratory Data Analysis

**Number of style images**

In [ ]:
len(style_images)

**Number of content images**

In [ ]:
len(content_images)

**Style Images**

In [ ]:
sample(style_images, 5)

**Content Images**

In [ ]:
sample(content_images, 5)

## Load Pretrained Model

In [ ]:
hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

## Generating outputs

In [ ]:
%rm -r -f /kaggle/working/images

In [ ]:
%mkdir /kaggle/working/images

In [ ]:
from tqdm import tqdm
for i in tqdm(range(num_outputs)):
    content_image_path = content_images[np.random.randint(len(content_images))]
    style_image_path = style_images[np.random.randint(len(style_images))]
    content_image = plt.imread(content_image_path)
    style_image = plt.imread(style_image_path)
    content_image = content_image.astype(np.float32)[np.newaxis, ...] / 255.
    style_image = style_image.astype(np.float32)[np.newaxis, ...] / 255.
    style_image = tf.image.resize(style_image, (image_size, image_size))
    outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
    stylized_image = tf.reshape(outputs[0], (image_size, image_size, 3))
    file_path = os.path.join(output_path, str(i + 1) + ".jpg")
    img = Image.fromarray(np.uint8(stylized_image * 255.0))
    img.save(file_path)

In [ ]:
shutil.make_archive(output_path, 'zip', output_path)

## Visualize outputs

In [ ]:
output_images = np.array([os.path.join(output_path, path) for path in os.listdir(output_path)])

In [ ]:
sample(output_images, 5)

## If this notebook is helpful, please give me an upvote, thank you.